<a href="https://colab.research.google.com/github/AleemRahil/Robust-End-to-End-E-Commerce-Analytics-Automation-with-LLMs/blob/main/LLM_knowledge_bases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Building LLM Knowledge Bases For Advanced SQL Chains

In this project, I explored how to build knowledge bases for advanced SQL generation using LangChain.

We'll leverage LangChain chains to extract insights from BigQuery by generating and executing SQL queries. Redis will store the SQL query examples needed for few-shot-prompting.

Organizations constantly seek ways to extract valuable insights from their data warehouses. However, writing complex SQL queries to fetch database information can be time-consuming and require specialized expertise.

But now, we can leverage LLMs to generate the SQL needed to extract insights. We can generate sophisticated queries based on natural language input using SQL knowledge bases and few-shot-prompting techniques. This approach empowers non-technical users to access and analyze data effectively, democratizing data exploration and insights.

In this project, we'll leverage LangChain chains to extract insights from BigQuery. To store our SQL query examples, we'll utilize Redis, a fast and efficient in-memory data store. Redis will serve as our knowledge base, allowing us to store and retrieve example queries. By leveraging Redis, we can perform similarity searches to find relevant query examples based on user input, enhancing the accuracy and relevance of the generated SQL queries.

Throughout this notebook, we'll explore various components of LangChain, such as the SQL query chain, few-shot prompt templates, and semantic similarity example selectors. To illustrate the practical application of our knowledge base, we'll tackle a common challenge in fetching analytics data from BigQuery: dealing with nested data using the UNNEST function.

In [ ]:
!pip install -qU \
  python-dotenv \
  langchain \
  openai \
  anthropic \
  langchain-openai \
  langchain-anthropic \
  sentence-transformers\
  sqlalchemy \
  sqlalchemy-bigquery \
  redis \
  google-cloud-bigquery

#Connecting to BigQuery


In [ ]:
from langchain.sql_database import SQLDatabase

project="rabbitpromotion"
dataset="gbq_chat"

service_account_path = './gbqkey.json'

url = f'bigquery://{project}/{dataset}?credentials_path={service_account_path}'

db = SQLDatabase.from_uri(url)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

#Initializing Language Models


In [ ]:
from langchain_anthropic import ChatAnthropic
llm_claude3 = ChatAnthropic(model='claude-3-opus-20240229')

from langchain_openai import ChatOpenAI
llm_gpt4 = ChatOpenAI(model="gpt-4")

#Creating a SQL Query Chain


In [ ]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm_gpt4, db)


In [ ]:
response = chain.invoke({"question": "How many customers are in the dataset?"})
print(response)

#Setting Up Redis


In [ ]:
REDIS_URL="your_redis_url"
REDIS_HOST="your_redis_host"
REDIS_PASSWORD="your_redis_password"
REDIS_PORT="your_redis_port"

import os

os.environ["REDIS_URL"]=REDIS_URL

In [ ]:
import redis
from langchain.vectorstores.redis import Redis

r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)

In [ ]:
r.ping()

True

In [ ]:
r.flushdb()

True

#Defining SQL Query Examples

In [ ]:
examples=[{'input': 'Get the total amount spent by each customer, including their first and last name.',
  'query': '\n            SELECT \n                c.firstName,\n                c.lastName,\n                c.amountSpent.amount\n            FROM rabbitpromotion.gbq_chat.customers c\n        '},
 {'input': 'Find the number of orders placed by each customer, along with their email marketing consent status.',
  'query': '\n            SELECT\n                c.id,\n                c.firstName,\n                c.lastName,\n                c.emailMarketingConsent.marketingState AS emailMarketingConsentStatus,\n                c.numberOfOrders\n            FROM rabbitpromotion.gbq_chat.customers c\n        '},
 {'input': 'Get the details of the last order placed by each customer.',
  'query': '\n            SELECT\n                c.id,\n                c.firstName,\n                c.lastName,\n                c.lastOrder.id AS lastOrderId\n            FROM rabbitpromotion.gbq_chat.customers c\n        '},
 {'input': 'Find the total amount spent by customers who have consented to email marketing.',
  'query': "\n            SELECT\n                SUM(CAST(c.amountSpent.amount AS NUMERIC)) AS totalAmountSpent\n            FROM rabbitpromotion.gbq_chat.customers c\n            WHERE c.emailMarketingConsent.marketingState = 'SUBSCRIBED'\n        "},
 {'input': 'Get the total number of orders and total amount spent for each currency.',
  'query': '\n            SELECT\n                o.currencyCode,\n                COUNT(*) AS totalOrders,\n                SUM(CAST(c.amountSpent.amount AS NUMERIC)) AS totalAmountSpent\n            FROM rabbitpromotion.gbq_chat.orders o\n            JOIN rabbitpromotion.gbq_chat.customers c ON o.customer.id = c.id\n            GROUP BY o.currencyCode\n        '},
 {'input': 'Find the top 5 customers who have placed the most orders.',
  'query': '\n            SELECT\n                c.id,\n                c.firstName,\n                c.lastName,\n                c.numberOfOrders\n            FROM rabbitpromotion.gbq_chat.customers c\n            ORDER BY c.numberOfOrders DESC\n            LIMIT 5\n        '},
 {'input': 'Get the total quantity of each product sold.',
  'query': '\n            SELECT\n                p.title,\n                COUNT(*) AS totalQuantitySold\n            FROM rabbitpromotion.gbq_chat.orders o\n            CROSS JOIN UNNEST(o.lineItems) AS lineItem\n            JOIN rabbitpromotion.gbq_chat.products p ON lineItem.productId = p.id\n            GROUP BY p.title\n        '},
 {'input': 'Find the top 5 best-selling products based on the total quantity sold.',
  'query': '\n            SELECT\n                p.title,\n                COUNT(*) AS totalQuantitySold\n            FROM rabbitpromotion.gbq_chat.orders o\n            CROSS JOIN UNNEST(o.lineItems) AS lineItem\n            JOIN rabbitpromotion.gbq_chat.products p ON lineItem.productId = p.id\n            GROUP BY p.title\n            ORDER BY totalQuantitySold DESC\n            LIMIT 5\n        '},
 {'input': 'Get the total revenue generated by each product.',
  'query': '\n            SELECT\n                p.title,\n                SUM(CAST(c.amountSpent.amount AS NUMERIC)) AS totalRevenue\n            FROM rabbitpromotion.gbq_chat.orders o\n            CROSS JOIN UNNEST(o.lineItems) AS lineItem\n            JOIN rabbitpromotion.gbq_chat.products p ON lineItem.productId = p.id\n            JOIN rabbitpromotion.gbq_chat.customers c ON o.customer.id = c.id\n            GROUP BY p.title\n        '},
 {'input': 'Find the average order value for each customer.',
  'query': '\n            SELECT\n                c.id,\n                c.firstName,\n                c.lastName,\n                AVG(CAST(c.amountSpent.amount AS NUMERIC)) AS averageOrderValue\n            FROM rabbitpromotion.gbq_chat.customers c\n            GROUP BY c.id, c.firstName, c.lastName\n        '},
 {'input': 'Get the total number of orders placed by customers with each email marketing consent status.',
  'query': '\n            SELECT\n                c.emailMarketingConsent.marketingState AS emailMarketingConsentStatus,\n                COUNT(*) AS totalOrders\n            FROM rabbitpromotion.gbq_chat.customers c\n            JOIN rabbitpromotion.gbq_chat.orders o ON c.id = o.customer.id\n            GROUP BY c.emailMarketingConsent.marketingState\n        '},
 {'input': 'Find the products purchased by each customer in their last order.',
  'query': '\n            SELECT\n                c.id AS customerId,\n                c.firstName,\n                c.lastName,\n                p.title AS productTitle\n            FROM rabbitpromotion.gbq_chat.customers c\n            JOIN rabbitpromotion.gbq_chat.orders o ON c.lastOrder.id = o.id\n            CROSS JOIN UNNEST(o.lineItems) AS lineItem\n            JOIN rabbitpromotion.gbq_chat.products p ON lineItem.productId = p.id\n        '},
 {'input': 'Get the total quantity sold for each product variant.',
  'query': '\n            SELECT\n                p.variantTitle,\n                COUNT(*) AS totalQuantitySold\n            FROM rabbitpromotion.gbq_chat.orders o\n            CROSS JOIN UNNEST(o.lineItems) AS lineItem\n            JOIN rabbitpromotion.gbq_chat.products p ON lineItem.variantId = p.variantId\n            GROUP BY p.variantTitle\n        '},
 {'input': 'Find the top 3 product variants with the highest total revenue.',
  'query': '\n            SELECT\n                p.variantTitle,\n                SUM(CAST(c.amountSpent.amount AS NUMERIC)) AS totalRevenue\n            FROM rabbitpromotion.gbq_chat.orders o\n            CROSS JOIN UNNEST(o.lineItems) AS lineItem\n            JOIN rabbitpromotion.gbq_chat.products p ON lineItem.variantId = p.variantId\n            JOIN rabbitpromotion.gbq_chat.customers c ON o.customer.id = c.id\n            GROUP BY p.variantTitle\n            ORDER BY totalRevenue DESC\n            LIMIT 3\n        '},
 {'input': 'Get the number of customers with each tag.',
  'query': '\n            SELECT\n                tag,\n                COUNT(*) AS numberOfCustomers\n            FROM rabbitpromotion.gbq_chat.customers c\n            CROSS JOIN UNNEST(c.tags) AS tag\n            GROUP BY tag\n        '},
 {'input': 'Find the total amount spent by customers with each tag.',
  'query': '\n            SELECT\n                tag,\n                SUM(CAST(c.amountSpent.amount AS NUMERIC)) AS totalAmountSpent\n            FROM rabbitpromotion.gbq_chat.customers c\n            CROSS JOIN UNNEST(c.tags) AS tag\n            GROUP BY tag\n        '},
 {'input': 'Get the average number of orders placed by customers with each tag.',
  'query': '\n            SELECT\n                tag,\n                AVG(c.numberOfOrders) AS averageNumberOfOrders\n            FROM rabbitpromotion.gbq_chat.customers c\n            CROSS JOIN UNNEST(c.tags) AS tag\n            GROUP BY tag\n        '},
 {'input': 'Find the products purchased by customers with a specific tag.',
  'query': "\n            SELECT\n                p.title AS productTitle,\n                COUNT(*) AS totalQuantitySold\n            FROM rabbitpromotion.gbq_chat.customers c\n            CROSS JOIN UNNEST(c.tags) AS tag\n            JOIN rabbitpromotion.gbq_chat.orders o ON c.id = o.customer.id\n            CROSS JOIN UNNEST(o.lineItems) AS lineItem\n            JOIN rabbitpromotion.gbq_chat.products p ON lineItem.productId = p.id\n            WHERE tag = 'VIP'\n            GROUP BY p.title\n        "},
 {'input': 'Get the total revenue generated by customers with each email marketing consent status.',
  'query': '\n            SELECT\n                c.emailMarketingConsent.marketingState AS emailMarketingConsentStatus,\n                SUM(CAST(c.amountSpent.amount AS NUMERIC)) AS totalRevenue\n            FROM rabbitpromotion.gbq_chat.customers c\n            GROUP BY c.emailMarketingConsent.marketingState\n        '},
 {'input': 'Find the top 5 customers who have generated the highest total revenue.',
  'query': '\n            SELECT\n                c.id,\n                c.firstName,\n                c.lastName,\n                SUM(CAST(c.amountSpent.amount AS NUMERIC)) AS totalRevenue\n            FROM rabbitpromotion.gbq_chat.customers c\n            GROUP BY c.id, c.firstName, c.lastName\n            ORDER BY totalRevenue DESC\n            LIMIT 5\n        '},
 {'input': 'Get the number of customers who have placed orders in each month.',
  'query': '\n            SELECT\n                EXTRACT(MONTH FROM o.createdAt) AS month,\n                COUNT(DISTINCT o.customer.id) AS numberOfCustomers\n            FROM rabbitpromotion.gbq_chat.orders o\n            GROUP BY month\n        '}]


#Storing Examples in Redis

In [ ]:
for example in examples:
    print(f"Input: {example['input']}\nQuery: {example['query']}\n")

Input: Get the total amount spent by each customer, including their first and last name.
Query: 
            SELECT 
                c.firstName,
                c.lastName,
                c.amountSpent.amount
            FROM rabbitpromotion.gbq_chat.customers c
        

Input: Find the number of orders placed by each customer, along with their email marketing consent status.
Query: 
            SELECT
                c.id,
                c.firstName,
                c.lastName,
                c.emailMarketingConsent.marketingState AS emailMarketingConsentStatus,
                c.numberOfOrders
            FROM rabbitpromotion.gbq_chat.customers c
        

Input: Get the details of the last order placed by each customer.
Query: 
            SELECT
                c.id,
                c.firstName,
                c.lastName,
                c.lastOrder.id AS lastOrderId
            FROM rabbitpromotion.gbq_chat.customers c
        

Input: Find the total amount spent by customer

In [ ]:
sql_vector_data = [" ".join(example.values()) for example in examples]
sql_vector_data[0]

'Get the total amount spent by each customer, including their first and last name. \n            SELECT \n                c.firstName,\n                c.lastName,\n                c.amountSpent.amount\n            FROM rabbitpromotion.gbq_chat.customers c\n        '

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
rds = Redis.from_texts(sql_vector_data, embeddings,  index_name='sql_idx')

In [ ]:
results = rds.similarity_search("unnest", 5)
results

[Document(page_content='Get the total quantity sold for each product variant. \n            SELECT\n                p.variantTitle,\n                COUNT(*) AS totalQuantitySold\n            FROM rabbitpromotion.gbq_chat.orders o\n            CROSS JOIN UNNEST(o.lineItems) AS lineItem\n            JOIN rabbitpromotion.gbq_chat.products p ON lineItem.variantId = p.variantId\n            GROUP BY p.variantTitle\n        ', metadata={'id': 'doc:sql_idx:24d4c6399c754ebd94f0b2fc7f972407'}),
 Document(page_content='Find the top 3 product variants with the highest total revenue. \n            SELECT\n                p.variantTitle,\n                SUM(CAST(c.amountSpent.amount AS NUMERIC)) AS totalRevenue\n            FROM rabbitpromotion.gbq_chat.orders o\n            CROSS JOIN UNNEST(o.lineItems) AS lineItem\n            JOIN rabbitpromotion.gbq_chat.products p ON lineItem.variantId = p.variantId\n            JOIN rabbitpromotion.gbq_chat.customers c ON o.customer.id = c.id\n            

#Selecting Examples Using Semantic Similarity

In [ ]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    rds,
    k=5,
    input_keys=["input"],
)

In [ ]:
example_selector.select_examples({"input": "unnest"})

[{'id': 'doc:803db85eaeb7400c955ec433ebd5725e:52ad63b622f64ec79c1d73fc057973e5',
  'input': 'Get the details of the last order placed by each customer.',
  'query': '\n            SELECT\n                c.id,\n                c.firstName,\n                c.lastName,\n                c.lastOrder.id AS lastOrderId\n            FROM rabbitpromotion.gbq_chat.customers c\n        '},
 {'id': 'doc:803db85eaeb7400c955ec433ebd5725e:2619f55e1d1b4a63a06ab398b6a67738',
  'input': 'Get the total quantity sold for each product variant.',
  'query': '\n            SELECT\n                p.variantTitle,\n                COUNT(*) AS totalQuantitySold\n            FROM rabbitpromotion.gbq_chat.orders o\n            CROSS JOIN UNNEST(o.lineItems) AS lineItem\n            JOIN rabbitpromotion.gbq_chat.products p ON lineItem.variantId = p.variantId\n            GROUP BY p.variantTitle\n        '},
 {'id': 'doc:803db85eaeb7400c955ec433ebd5725e:751d4d42146c448293c157ec0366ce76',
  'input': 'Get the total

#Creating a Few-Shot Prompt Template


In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="""
    You are a  BigQuery SQL expert. Given an input question, create a syntactically correct BigQuery SQL query to run.
    Here is the relevant table info: {table_info}
    Unless otherwise specificed, do not return more than {top_k} rows
    Below are a number of examples of questions and their corresponding SQL queries.""",
    suffix="User input: {input}\nSQL query: ",
    input_variables=["input", "top_k", "table_info"],
)

#Fetching BigQuery Schema

In [ ]:
def build_schema_desc(fields, prefix=""):
    """Build schema description, including nested fields."""
    desc = []
    for f in fields:
        d = f"{prefix}- Name: {f.name}, Type: {f.field_type}, Mode: {f.mode}"
        desc.append(d)
        if f.field_type == 'RECORD':
            sub_desc = build_schema_desc(f.fields, prefix + "    ")
            desc.extend(sub_desc)
    return desc


def fetch_schemas(dataset_id, client):
    """Fetch schema descriptions for all tables in a dataset."""
    schemas = []
    simple_table_list = []

    tables = client.list_tables(dataset_id)
    for table in tables:
        ref = client.get_table(table)

        simple_table_list.append(f"- {ref.project}.{ref.dataset_id}.{ref.table_id}")

        schema_desc = [f"Schema for {table.table_id}:"]
        schema_desc += build_schema_desc(ref.schema)
        schema_desc.append("")  # For newline

        schemas += schema_desc

    return "\n".join(simple_table_list) + "\n\n" + "\n".join(schemas)

#Formatting the Prompt

In [ ]:
full_dataset_id = "rabbitpromotion.gbq_chat"

from google.cloud import bigquery
from google.oauth2 import service_account

service_account_path = './gbqkey.json'

credentials = service_account.Credentials.from_service_account_file(service_account_path)
gbq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

gbq_schema=fetch_schemas(full_dataset_id,gbq_client)

In [ ]:
print(gbq_schema)

- rabbitpromotion.gbq_chat.customers
- rabbitpromotion.gbq_chat.orders
- rabbitpromotion.gbq_chat.products

Schema for customers:
- Name: id, Type: STRING, Mode: REQUIRED
- Name: emailMarketingConsent, Type: RECORD, Mode: NULLABLE
    - Name: consentUpdatedAt, Type: TIMESTAMP, Mode: NULLABLE
    - Name: marketingOptInLevel, Type: STRING, Mode: NULLABLE
    - Name: marketingState, Type: STRING, Mode: NULLABLE
- Name: createdAt, Type: TIMESTAMP, Mode: NULLABLE
- Name: updatedAt, Type: TIMESTAMP, Mode: NULLABLE
- Name: firstName, Type: STRING, Mode: NULLABLE
- Name: lastName, Type: STRING, Mode: NULLABLE
- Name: numberOfOrders, Type: INTEGER, Mode: NULLABLE
- Name: amountSpent, Type: RECORD, Mode: NULLABLE
    - Name: amount, Type: STRING, Mode: NULLABLE
    - Name: currencyCode, Type: STRING, Mode: NULLABLE
- Name: lastOrder, Type: RECORD, Mode: NULLABLE
    - Name: id, Type: STRING, Mode: NULLABLE
- Name: tags, Type: STRING, Mode: REPEATED

Schema for orders:
- Name: id, Type: STRING, M

In [ ]:
print(prompt.format(input="How many customers are in the dataset?", top_k=5, table_info=gbq_schema))


    You are a  BigQuery SQL expert. Given an input question, create a syntactically correct BigQuery SQL query to run.
    Here is the relevant table info: - rabbitpromotion.gbq_chat.customers
- rabbitpromotion.gbq_chat.orders
- rabbitpromotion.gbq_chat.products

Schema for customers:
- Name: id, Type: STRING, Mode: REQUIRED
- Name: emailMarketingConsent, Type: RECORD, Mode: NULLABLE
    - Name: consentUpdatedAt, Type: TIMESTAMP, Mode: NULLABLE
    - Name: marketingOptInLevel, Type: STRING, Mode: NULLABLE
    - Name: marketingState, Type: STRING, Mode: NULLABLE
- Name: createdAt, Type: TIMESTAMP, Mode: NULLABLE
- Name: updatedAt, Type: TIMESTAMP, Mode: NULLABLE
- Name: firstName, Type: STRING, Mode: NULLABLE
- Name: lastName, Type: STRING, Mode: NULLABLE
- Name: numberOfOrders, Type: INTEGER, Mode: NULLABLE
- Name: amountSpent, Type: RECORD, Mode: NULLABLE
    - Name: amount, Type: STRING, Mode: NULLABLE
    - Name: currencyCode, Type: STRING, Mode: NULLABLE
- Name: lastOrder, Type: R

#Creating a SQL Query Chain with the Prompt


In [ ]:
chain = create_sql_query_chain(llm_gpt4, db, prompt)
chain.invoke({"question": "how many customers are there?"})

#Alternative Prompt Template


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")
prompt_alternative = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="""
    You are a  BigQuery SQL expert. Given an input question, create a syntactically correct BigQuery SQL query to run.
    Here is the relevant table info: {schema}
    Below are a number of examples of questions and their corresponding SQL queries.""",
    suffix="User input: {input}\nSQL query: ",
    input_variables=["input", "schema"],
)

#Creating a SQL Response Pipeline


In [ ]:
def get_schema(_):
    return fetch_schemas(full_dataset_id,gbq_client)

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt_alternative
    | llm_gpt4.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

#Generating SQL Queries


In [ ]:
sql_prompt="""
How many customers are there?
"""
res=sql_response.invoke({"input": sql_prompt})
print(res)

SELECT COUNT(*) AS totalCustomers
FROM rabbitpromotion.gbq_chat.customers


In [ ]:
gbq_client.query(res).to_dataframe()

,totalCustomers
0,14


In [ ]:
sql_prompt="""
List the 5 customers with highest spend
"""
res=sql_response.invoke({"input": sql_prompt})
gbq_client.query(res).to_dataframe()

,id,firstName,lastName,totalSpend
0,gid://shopify/Customer/6056367554809,Cedric,Cochran,469.460000000
1,gid://shopify/Customer/6056367751417,Stephen,Long,376.680000000
2,gid://shopify/Customer/6056367620345,Brennan,Lynch,359.370000000
3,gid://shopify/Customer/6056367849721,Connor,Ware,266.960000000
4,gid://shopify/Customer/6727568752889,s,d,264.310000000
